In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
from src.utils import get_project_root_path
import src.config as config

/Users/finnlueth/Developer/gits/prottrans-t5-signalpeptide-prediction/.venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [4]:
ROOT = get_project_root_path()

In [5]:
directory = ROOT + '/data/raw/split_4/'

In [6]:
dirs = os.listdir(directory)
tsv_files = [f for f in dirs if f.startswith('output_') and f.endswith('.txt')]

In [7]:
def process_label(label: str) -> str:
    result = [*label]
    for i, char in enumerate(result):
        if char == 'c':
            result[i] = result[i-1]
    return ''.join(result)

In [11]:
results = {}
for file in tsv_files:
    df_head = pd.read_csv(directory + file, sep='\t', nrows=0)
    df_body = pd.read_csv(directory + file, sep='\t', skiprows=1)
    label = ''.join(df_body.label.to_list())
    label = process_label(label)
    results.update({df_head.columns[0].replace('# Name=', '').split('|')[0]: label})

In [8]:
ground_truth = pd.read_parquet(ROOT + '/data/processed/5.0_train_full.parquet.gzip')

In [9]:
ground_truth.Label = ground_truth.Label.apply(lambda x: ''.join(config.label_decoding[y] for y in x))
ground_truth.Sequence = ground_truth.Sequence.apply(lambda x: x.replace(' ', ''))

In [12]:
df_final = pd.DataFrame.from_dict(results, orient='index', columns=['SignalP6_predicted_labels']).join(ground_truth.set_index('Uniprot_AC'))

In [ ]:
# df_final[['SignalP6_predicted_labels', 'Label']].head()

In [25]:
df_final.head(None)

,SignalP6_predicted_labels,Kingdom,Type,Partition_No,Sequence,Label,Split
O99019,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO,EUKARYA,NO_SP,4,MASISSLNQIPCKTLQITSQYSKPTSKISTLPISSTNFLSKTEQHRSISVKEFTNPKPKFTAQATNYDKE,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII,4
P02739,NNHHHHHHHHHHHHCCCCOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO,EUKARYA,SP,4,MKLFTGLIFCSLVLGVSSQWYSFIGEAVQGAWDMYRAYSDMREANYKNSDKYFHARGNYDAAQRGPGGAW,SSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO,4
P43319,NNNHHHHHHHHHHHHCCCCCOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO,NEGATIVE,LIPO,4,MKRAPLITGLLLISTSCAYASSGGCGADSTSGATNYSSVVDDVTVNQTDNVTGREFTSATLSSTNWQYAC,LLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO,4
Q8BUN5,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO,EUKARYA,NO_SP,4,MSSILPFTPPIVKRLLGWKKGEQNGQEEKWCEKAVKSLVKKLKKTGQLDELEKAITTQNVNTKCITIPRS,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII,4
A7MLE5,NNNRRNHHHHHHHHHHHHHHHHHHHHHCCCCCOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO,NEGATIVE,TAT,4,MNRRRFIKASLALAAACGTPGLATLFSRNAWAQESDIADGQTRAFDFSVLQAMASELAQKPWGGAPRPLP,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO,4
...,...,...,...,...,...,...,...
Q9UTN4,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO,EUKARYA,NO_SP,4,MERAENARIIQKPMTRRTVDYGSGLSKYIVNRHLRSNRYHIHVPRPNPNQIINLYPPYEYKYNNTSSLCT,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII,4
Q86W10,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO,EUKARYA,NO_SP,4,MEPSWLQELMAHPFLLLILLCMSLLLFQVIRLYQRRRWMIRALHLFPAPPAHWFYGHKEFYPVKEFEVYH,IIIIIIIIIMMMMMMMMMMMMMMMMMMMMMOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO,4
Q6Q6R3,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO,EUKARYA,NO_SP,4,MSWTCPRCQQPVYFAEKVSSLGKNWHRFCLKCERCHSILSPGGHAEHNGRPYCHKPCYGALFGPRGVNIG,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII,4
Q9BDB7,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO,EUKARYA,NO_SP,4,MKVTARLTWIEEKILEKLLGNASLTLLYQSSAHKNCVSEMTQKYSLQGSTMTVFHLEKDVVVGVFILENF,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII,4


In [ ]:
df_final.to_parquet(ROOT + '/data/processed/df_SignalP6_predicted_labels.parquet.gzip', compression='gzip')

---